In [2]:
import pandas as pd
import numpy as np
import os
from scipy.cluster import hierarchy as hc
import matplotlib
import matplotlib.pyplot as plt
import warnings
import seaborn as sns
from scipy import stats
import math
import sys
import xlsxwriter
from datetime import datetime, timedelta


%matplotlib inline
warnings.filterwarnings(action='ignore')

matplotlib.rcParams['font.family'] ='Malgun Gothic'
matplotlib.rcParams['axes.unicode_minus'] =False

# Load data

In [3]:
dir_root = 'C:/Users/10123/Desktop/part2'
dir_master = os.path.join(dir_root, '01. Data', '00. master_data')
dir_processed = os.path.join(dir_root, '01. Data', '01. 재무지표1', 'ITS')
dir_result = os.path.join(dir_root, '04. 상관분석', '01. 재무지표1', '02. result')

# Moving corr

In [5]:
master_file_name = 'desc_v2.2.xlsx'
version_processed = 'v1.3'

### Load data, set parameter

In [6]:
master = pd.read_excel(os.path.join(dir_master, master_file_name), sheet_name='desc')
master = master.drop('Unnamed: 0', axis=1)

In [8]:
y = 'close'
date = 'Date_x'
r_window_size = 20

company = ['Accenture', 'Cognizant', 'Infosys']


In [12]:
# 20 point의 마지막 지점을 column으로

writer = pd.ExcelWriter(os.path.join(dir_result, 'Dynamic_correlation_'+y+'.xlsx'), engine='xlsxwriter')
    
for i in company:
    df = pd.read_csv(os.path.join(dir_processed, i+'_non_shift_clean_'+version_processed+'.csv'))
    df[date] = pd.to_datetime(df[date])
    col = [x for x in df.columns if (x not in [date, y, 'close_weighted', 'EV_EBITDA', 'PBR'])]
    
    if df.isna().sum().sum() != 0:
        print(df.shape)
        print('NaN exists')
        break
    
    tmp = df['Date_x']
    
    for j in col:
        tmp= pd.concat([tmp, 
                        pd.Series(df[y].rolling(window=r_window_size).corr(df[j]), name=j)],
                        axis=1)
    tmp = tmp[~tmp['매출액(천원)'].isna()]
#     tmp = tmp[(tmp[date].dt.month >= 10) & (tmp[date].dt.month <= 12)]
    tmp = pd.merge(tmp, df[[date, y]], on=date, how='left')
    tmp = tmp.iloc[tmp[date].dt.year.drop_duplicates(keep='last').index]
    #     display(tmp)
    tmp = tmp.set_index('Date_x').transpose().round(3)
    tmp.columns = tmp.columns.astype('str')
    tmp = tmp.reset_index()
    tmp = tmp.rename({'index':'variable'}, axis=1)

    tmp = pd.merge(tmp, master[['variable','group','importance']], on='variable', how='left')
    tmp = tmp.set_index(['variable', 'group', 'importance'])
    
    tmp.to_excel(writer, sheet_name=i, index=True)

writer.save()
